In [86]:
import keras
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import misc
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
import pickle
%matplotlib inline

In [87]:
test_img_fnames = os.listdir('data/data/test')
train_img_fnames = os.listdir('data/data/train')

# Load in the data
First we need to write a function to load into memory each of the images, and then resize them to (224, 224, 3). scipy.misc.imread and scipy.misc.imresize will be useful for this. You should also rescale the images so that they are on a scale of 0 to 1, meaning each pixel value should be between 0 and 1. This can be achieved by dividing by an appropriate constant.

In [88]:
def load_img(filename):
    #load current image
    current_img = misc.imread(filename)
    #resize current image
    current_img = misc.imresize(current_img,(224,224))
    return current_img

In [89]:
test_imgs = [load_img(os.path.join('data/data/test', fname)) for fname in test_img_fnames]
test_imgs = np.stack(test_imgs)

In [90]:
train_imgs = [load_img(os.path.join('data/data/train', fname)) for fname in train_img_fnames]
train_imgs = np.stack(train_imgs)

In [91]:
with open('data/data/train_labels.pkl', 'rb') as f:
    train_labels = pickle.load(f)

# Load Pretrained VGG network
Now in order to generate the CNN featurizations of our images we need to load a pretrained network. Note that running this network will take a long time on CPU so you can feel free to skip this section and just load the featurizations I provided in the next section

In [ ]:
VGG16 = keras.applications.vgg16.VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

In [ ]:
VGG16.summary()

In [ ]:
featurized_training_data = VGG16.predict(train_imgs, verbose=1)

In [ ]:
featurized_test_data = VGG16.predict(test_imgs, verbose=1)

In [ ]:
# Save featurizations
import pickle
with open('featurized_train_imgs.pkl', 'wb') as f:
    pickle.dump(featurized_training_data, f)
with open('featurized_test_imgs.pkl', 'wb') as f:
    pickle.dump(featurized_test_data, f)

# Load featurizations

In [92]:
import pickle
with open('data/featurized_train_imgs.pkl', 'rb') as f:
    featurized_training_data = pickle.load(f)
with open('data/featurized_test_imgs.pkl', 'rb') as f:
    featurized_test_data = pickle.load(f)

In [96]:
featurized_training_data.

(10000, 7, 7, 512)

# Create you network
Now we need to create a network to take in the featurizations and output a label of dog or not dog. To do this you should use Keras' Sequential model. We will need to flatten our (7,7,512) feature input into a vector (HINT: lookup flatten in keras documentation) and then add a Dense layer with some number of neurons (play around with the number of neurons to improve your performance). Then finally we need a Dense layer with 1 neuron and a sigmoid activation to represent our label output. You might want to use more or less model.add calls than have been provided

In [98]:
model = Sequential()
model.add(Flatten(input_shape = (7, 7, 512)))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(16,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

In [99]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 16)                401424    
_________________________________________________________________
dense_11 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 17        
Total params: 401,713
Trainable params: 401,713
Non-trainable params: 0
_________________________________________________________________


# Now we need to train the network
You need to compile the model first, and then use the fit function. You should use binary crossentropy as your loss.

In [100]:
model.compile(optimizer = 'SGD' ,loss = 'binary_crossentropy')

In [104]:
model.fit(featurized_training_data,train_labels,epochs = 2)

Epoch 1/2
10000/10000 [==============================] - 4s 397us/step - loss: 0.5930
Epoch 2/2
10000/10000 [==============================] - 2s 229us/step - loss: 0.3924


Now we need to predict labels for the test set and output it to file. Use keras predict for this. Remember that the predictions are real values between 0 and 1 and you should be outputting just 0 or 1, not a value between.

In [105]:
predictions = model.predict(featurized_test_data)

In [108]:
predictions

array([[ 0.14544983],
       [ 0.88839579],
       [ 0.92664123],
       ..., 
       [ 0.83916843],
       [ 0.22811162],
       [ 0.85091788]], dtype=float32)

In [110]:
with open('test_labels.txt', 'w') as f:
    f.write("Id,Label\n")
    for i, val in enumerate(predictions):
        if val < 0.5:
            prediction = 0
        else:
            prediction = 1
        #prediction = 
        f.write(str(i))
        f.write(' -- ')
        #f.write('\n')
        f.write(str(prediction))
        f.write('\n')

# Part 2

In [ ]:
from sklearn.decomposition import PCA

Now we want to run PCA on our images and our featurizations to see the difference. To save time we will just run it on the first 500 images. Take the first 500 images and 500 featurizations and reshape them into shape (500, x).

In [ ]:
original_imgs = ???
featurized = ???

Run PCA with 2 components on the original images

In [ ]:
pca_orig = ???
# YOUR CODE HERE

Run PCA with 2 components on the featurizations

In [ ]:
pca_featurized = ???
# YOUR CODE HERE

Project the original images and featurizations onto the 2 principal components. (HINT: look at the PCA.transform function)

In [ ]:
projected_orig_imgs = ???
projected_featurized = ???

Find the indices of the labels that are cats and the indices that are dogs. np.where will make this very easy

In [ ]:
labels = train_labels[:500]
cat_inds = ???
dog_inds = ???

Plot the PCA of the original images and the PCA of the featurization

In [ ]:
plt.scatter(projected_orig_imgs[cat_inds ,0], projected_orig_imgs[cat_inds, 1], c='red')
plt.scatter(projected_orig_imgs[dog_inds, 0], projected_orig_imgs[dog_inds, 1], c='green')

In [ ]:
plt.scatter(projected_featurized[cat_inds ,0], projected_featurized[cat_inds, 1], c='red')
plt.scatter(projected_featurized[dog_inds, 0], projected_featurized[dog_inds, 1], c='green')